# [백준/앱](https://www.acmicpc.net/problem/7579)


## 풀이과정


### 첫번째 시도

단순 DP + 배낭 문제라고 생각하고 풀었는데 메모리 초과가 났다. 문제 조건을 다시 보니 $M$의 최댓값이 백만이었다. 이를 개선하기 위해서는 비교적 작은 값인 $C$를 기준으로 DP를 진행했어야 했다.


#### 풀이과정


In [ ]:
def solution():
    import sys

    input = sys.stdin.readline
    n, m = map(int, input().split())
    memories = [0] + [int(i) for i in input().split()]
    costs = [0] + [int(i) for i in input().split()]
    MAX = sum(memories) - m
    dp = [[0] * (MAX + 1) for _ in range(n + 1)]
    for i in range(1, n + 1):
        mi = memories[i]
        for j in range(mi, MAX + 1):
            dp[i][j] = max(dp[i - 1][j], dp[i - 1][j - mi] + costs[i])
    print(sum(costs) - dp[-1][-1])


solution()

### 두번째 시도

이번에는 $C$를 기준으로 DP를 진행했다. 하지만 이번에는 답이 오답이 나왔다. 다른 사람의 풀이와 비교해 봐도 다를 것이 없었는데 왜 틀렸는지 이해가 가질 않았다.


#### 풀이과정


In [ ]:
def solution():
    import sys

    input = sys.stdin.readline
    _, M = map(int, input().split())
    memories = list(map(int, input().split()))
    costs = list(map(int, input().split()))
    MAX = sum(costs) + 1
    dp = [0] * MAX

    for c, m in zip(costs, memories):
        prev, dp = dp, dp[:c]
        for j in range(c, MAX):
            dp.append(max(prev[j - c] + m, prev[j]))

    print(next(i for i, d in enumerate(dp) if d >= M))


solution()

### 세번째 시도

하지만 다른 사람을 코드를 잘 보니 내가 놓친 부분이 있었다. 나는 메모리를 위해 DP를 1차원 배열로 구현했는데, 알고 보니 값을 업데이트 할 때 이전 값이 필요했다. 다른 사람들은 2차원 배열로 구현해서 문제가 없었지만 나는 1차원 배열로 구현했기에 이미 업데이트 된 값을 참조해서 후속 값을 업데이트 한 것이었다. 이를 위해 이전 값을 `prev`라는 변수로 저장해서 사용하니 문제가 풀렸다.


#### 풀이과정


In [ ]:
def solution():
    import sys

    input = sys.stdin.readline
    _, M = map(int, input().split())
    memories = list(map(int, input().split()))
    costs = list(map(int, input().split()))
    MAX = sum(costs) + 1
    dp = [0] * MAX

    for c, m in zip(costs, memories):
        prev, dp = dp, dp[:c]
        for j in range(c, MAX):
            dp.append(max(prev[j - c] + m, prev[j]))

    print(next(i for i, d in enumerate(dp) if d >= M))


solution()

### 네번째 시도

내가 틀렸던 부분을 다시 생각해 보았다. 이전 배열을 참고하는 것은 맞지만 특정 인덱스를 업데이트 할 떄 이전의 배열에서 그 이전의 인덱스 값은 참조해도 이후의 값은 참조되지 않았다. 이를 이용해 다시 1차원 배열로 구현하면서 동시에 뒤의 값부터 업데이트 되도록 구현했다. 이런 방식으로 구현하니 더 좋은 결과를 얻어 PyPy3 1위라는 결과를 얻었다.


#### 풀이과정


In [ ]:
def solution():
    import sys

    input = sys.stdin.readline
    _, M = map(int, input().split())
    memories = list(map(int, input().split()))
    costs = list(map(int, input().split()))
    MAX = sum(costs)
    dp = [0] * (MAX + 1)

    for c, m in zip(costs, memories):
        for j in range(MAX, c - 1, -1):
            dp[j] = max(dp[j - c] + m, dp[j])

    print(next(i for i, d in enumerate(dp) if d >= M))


solution()

![2024년 2월 7일 10시 기준 7579번 맞힌 사람 (PyPy3) 112ms로 1위](<../../img/Screenshot 2024-02-07 at 10-41-54 7579번 맞힌 사람 (PyPy3) - 1 페이지.png>)


### 다섯번째 시도

기존의 풀이를 함수형과 반복자 위주로 바꿔보았다. 좋은 효율을 보이지는 않았지만 이런 방식으로도 풀 수 있음을 보여주고 싶었다.


#### 풀이과정


In [ ]:
from itertools import takewhile, dropwhile
from functools import reduce

add = lambda a: lambda b: a + b
lt = lambda M: lambda x: x < M
mapper = lambda update: lambda dp, mc: update(dp, *mc)
parse_int = lambda input: lambda: map(int, input().split())
list_lt = lambda M: lambda it: list(takewhile(lt(M), it))
next_nth = lambda n: lambda it: next(dropwhile(lt((n,)), enumerate(it)))[1]
zeros = lambda n: [0] * n


def solution():
    import sys

    input = parse_int(sys.stdin.readline)
    lt_M = list_lt(next_nth(1)(input()))
    update = lambda dp, m, c: add(dp[:c])(lt_M(map(max, map(add(m), dp), dp[c:])))
    print(len(list(reduce(mapper(update), zip(input(), input()), zeros(10001)))))


solution()

### 여섯번째 시도

[lambda님의 풀이](https://www.acmicpc.net/source/38360098)를 개선시켰더니 더 좋은 효율의 코드가 나와서 Python3 에서도 1위를 기록했다. Python 전체 순위는 랭크가 꼬였는지 오히려 더 느린 코드가 상위권으로 올라와 있었다.

Python에서는 변수 할당이 느리기 때문에 이를 최소화했다. 또 비용과 메모리를 정렬할 때 굳이 메모리까지 정렬할 필요가 없기 때문에 비용만 정렬했다. 마지막으로 메모리가 목표보다 커지면 더 이상 계산할 필요가 없기 때문에 생략해버렸다. 이를 통해 더 좋은 효율을 얻을 수 있었다.


#### 풀이과정


In [ ]:
# refer https://www.acmicpc.net/source/38360098

def solution():
    import sys

    input = sys.stdin.buffer.readline
    _, M = map(int, input().split())
    dp = {0: 0}
    for m, c in sorted(zip(map(int, input().split()), map(int, input().split())), reverse=True, key=lambda x: x[0]):
        dp.update({ccc: mmm for cc, mm in dp.items() if mm < M and dp.get((ccc := cc + c), 0) < (mmm := mm + m)})
    print(min((cc for cc, mm in dp.items() if mm >= M)))


solution()

![2024년 2월 12일 5시 기준 7579번 맞힌 사람 (Python3) 64ms로 1위](<../../img/Screenshot 2024-02-12 at 05-23-44 7579번 맞힌 사람 (Python 3) - 1 페이지.png>)


## 해답


In [1]:
def solution():
    import sys

    input = sys.stdin.readline
    _, M = map(int, input().split())
    dp = {0: 0}
    for m, c in sorted(zip(map(int, input().split()), map(int, input().split())), reverse=True):
        dp.update({ccc: mmm for cc, mm in dp.items() if mm < M and dp.get((ccc := cc + c), 0) < (mmm := mm + m)})
    print(min((cc for cc, mm in dp.items() if mm >= M)))

## 예제


In [2]:
# 백준 문제 풀이용 예제 실행 코드
from bwj import test

test_solution = test(solution)

# test_solution("""""")
# test_solution(read("fn").read())

In [3]:
test_solution(  # 3
    """3 50
10 40 30
0 3 2
"""
)

3


In [4]:
test_solution(  # 0
    """5 60
30 10 20 35 40
0 1 0 0 0
"""
)

0


In [5]:
test_solution(  # 1
    """5 60
20 30 10 20 30
0 0 1 2 3
"""
)

1


In [6]:
test_solution(  # 6
    """5 60
30 10 20 35 40
3 0 3 5 4
"""
)

6
